Lets calculate:
$$
\begin{align*}
    
\partial_{t}(E[ 5U^{2}(0,T)-T^{2}])&=\partial_{t}(E[ 5(U(0,1)T)^{2}-T^{2}]) \\
&=E[ \partial_{t}(5(U(0,1)T)^{2}-T^{2})] \\
&=E[ 10(U(0,1)T)^{2}/T-2T]\\
&=E[ 10U^{2}(0,T)/T-2T].

\end{align*}
$$ 


In [7]:
from random import random

def X(T): return 5*random()**2*T**2 -T**2 
def dX(T): return 10*random()**2*T -2*T 

nsim = 10**5
T=1
dT = 0.05
dx = sum(dX(T) for _ in range(nsim))/nsim
dxx = sum((X(T+dT) - X(T))/dT for _ in range(nsim))/nsim


print(dx)
print(dxx)

1.3265042696078901
1.5658517940157437


Lets now look at:
$$
\partial_{t} E[U^{t}(0,1)].
$$ 


In [11]:
from random import random
from math import pow,log

def Z(t): return pow(random(),t)

def dZ(t): 
    u = random()
    return log(u)*pow(u,t)

nsim = 10**5
t=1
dt = 0.05
dz = sum(dZ(t) for _ in range(nsim))/nsim
dzz = sum((Z(t+dt) - Z(t))/dt for _ in range(nsim))/nsim

print(dz)
print(dzz)

-0.25039363417337124
-0.24476202748682982


Lets now look at something more discrete:

$$
\begin{align*}
X(p)&= 2 \bold{1}_{\{U(0,1)<p\}} -1 \\
\partial_{p}X(p)&= 2 \delta(U(0,1)-p)
\end{align*}
$$

If you calculate the expectances it works out ...


Lets do a RMC example:
$$
\begin{align*}
Y(t)&= 1 + t Y(Ut) \\
\partial_{t}Y(t)&= t U \partial_{t}Y(Ut) + Y(Ut)
\end{align*}
$$ 

we implement this with russian roulette:

In [181]:
from random import random
import numpy as np

def X(t):
    if random()>t: return np.array((1,0))
    U = random()
    ys,dys = X(U*t)
    yt = 1 + ys
    dyt = ys/t + U*dys
    return np.array((yt,dyt))

nsim = 10**5
sol = sum( X(0.5) for _ in range(nsim))/nsim
print(sol)

[1.65069 1.65746]


Another RMC example:
$$
\begin{align*}
Y(t,a)&= 1 + ta Y(Ut,a) \\
\partial_{a}Y(t,a)&= t (Y(Ut,a) + a \partial_{a}Y(Ut,a) )
\end{align*}
$$ 

Remember that $E(t,a) = e^{ta}$. Lets implement this:

In [46]:
from random import random
from math import exp
import numpy as np

def X2(t,a):
    if random()>t: return np.array((1,0)) 
    U = random()
    ys,days = X2(U*t,a) 
    yt = 1 + a*ys*(1 if t<1 else t)
    dayt = (ys + a*days)*(1 if t<1 else t)
    return np.array((yt,dayt))

def test():
    t, a, nsim = 1.2, 2, 10**3 
    s = sum(X2(t, a) for _ in range(nsim))/nsim
    sol = np.array([exp(t*a), t*exp(t*a)])
    err = (s-sol)/sol
    print(f"X2 = {s}", f"%error is {err}")

test()

X2 = [11.37528334 14.34584559] %error is [0.03194242 0.08452146]


Actually this was the same expression we got by differentiating the integral equation.